In [ ]:
import argparse
import torch
import caramel

In [ ]:
def set_args():
    if args_parser is not None:
        args = args_parser
    else:
        args = dotdict()
        args.seed = 1
        args.log_interval = 10    
        args.batch_size = 100
        args.epochs = 1
        args.with_cuda = False
        args.chkpt_interval = 10
        args.isambard = False
        args.warm_start = False
        args.identifier = '021501AQ1H'
        args.data_region = '021501AQ1H'
        args.normaliser = 'standardise_mx'
        args.loss = 'mae'
        args.nb_hidden_layers = 9
        args.nlevs = 30
        args.data_fraction = 0.01

    torch.manual_seed(args.seed)
    args.cuda = args.with_cuda and torch.cuda.is_available()
    args.device = torch.device("cuda" if args.cuda else "cpu")
    kwargs = {'pin_memory': False} if args.cuda else {}

    # Define the Model
    # n_inputs,n_outputs=140,70
    args.region=args.data_region
    args.in_features = (args.nlevs*4+3)
    args.nb_classes =(args.nlevs*2)
    args.hidden_size = int(0.66 * args.in_features + args.nb_classes)
    args.model_name = "q_qadv_t_tadv_swtoa_lhf_shf_qtnext_{0}_lyr_{1}_in_{2}_out_{3}_hdn_{4}_epch_{5}_btch_{6}_{7}_{8}_levs.tar".format(str(args.nb_hidden_layers).zfill(3),
                                                                                    str(args.in_features).zfill(3),
                                                                                    str(args.nb_classes).zfill(3),
                                                                                    str(args.hidden_size).zfill(4),
                                                                                    str(args.epochs).zfill(3),
                                                                                    str(args.batch_size).zfill(5),
                                                                                    args.identifier, 
                                                                                    args.loss,
                                                                                    args.normaliser)

    # Get the data
    if args.isambard:
        args.locations={ "train_test_datadir":"/home/mo-ojamil/ML/CRM/data",
                "chkpnt_loc":"/home/mo-ojamil/ML/CRM/data/models/chkpts",
                "hist_loc":"/home/mo-ojamil/ML/CRM/data/models",
                "model_loc":"/home/mo-ojamil/ML/CRM/data/models/torch",
                "normaliser_loc":"/home/mo-ojamil/ML/CRM/data/normaliser/{0}_{1}".format(args.region, args.normaliser)}
    else:
        args.locations={ "train_test_datadir":"/project/spice/radiation/ML/CRM/data/models/datain",
                "chkpnt_loc":"/project/spice/radiation/ML/CRM/data/models/chkpts/torch",
                "hist_loc":"/project/spice/radiation/ML/CRM/data/models/history",
                "model_loc":"/project/spice/radiation/ML/CRM/data/models/torch",
                "normaliser_loc":"/project/spice/radiation/ML/CRM/data/models/normaliser/{0}_{1}".format(args.region, args.normaliser)}
    return args


In [ ]:
args = set_args()
model, loss_function, optimizer, scheduler = caramel.set_model(args)
caramel.train_loop(model, loss_function, optimizer, scheduler, args)